# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [64]:
weather_data=pd.read_csv("../WeatherPy/Output/weather_data.csv", encoding="ISO-8859-1")
weather_data

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,46.40,66,90,11.50,CL,1610327488
1,Tiznit Province,29.5833,-9.5000,46.92,83,95,6.42,MA,1610327489
2,Saskylakh,71.9167,114.0833,-8.48,87,100,16.80,RU,1610327490
3,Ushuaia,-54.8000,-68.3000,42.80,70,75,20.71,AR,1610327317
4,Nikolskoye,59.7035,30.7861,3.99,78,0,8.95,RU,1610327493
...,...,...,...,...,...,...,...,...,...
589,Buarcos,40.1660,-8.8768,44.01,69,0,4.61,PT,1610328946
590,AswÄn,24.0934,32.9070,61.14,60,0,8.81,EG,1610328948
591,Panguna,-6.3164,155.4848,80.85,76,94,1.77,PG,1610328949
592,San QuintÃ­n,30.4833,-115.9500,60.15,59,3,7.56,MX,1610328950


### Humidity Heatmap


In [68]:
#Configure gmaps.
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Add Heatmap layer to map.
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_data['Humidity'].max(),
                                 point_radius=3)


# Add layer
fig.add_layer(humidity_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [69]:
weather_data.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,46.40,66,90,11.50,CL,1610327488
1,Tiznit Province,29.5833,-9.5000,46.92,83,95,6.42,MA,1610327489
2,Saskylakh,71.9167,114.0833,-8.48,87,100,16.80,RU,1610327490
3,Ushuaia,-54.8000,-68.3000,42.80,70,75,20.71,AR,1610327317
4,Nikolskoye,59.7035,30.7861,3.99,78,0,8.95,RU,1610327493


In [70]:
#ideal weather conditions: maximum temperature between 59 to 86(F)
                         # wind speed <= 5(mph)
                         # humidity < 50 (%), cloudiness =0
idealvacation_df = weather_data.loc[(weather_data['Max temp']>=55) & (weather_data['Max temp']<=86) & 
                                    (weather_data['Humidity']<=50)& (weather_data['Cloudiness']<=0)
                                    & (weather_data['Wind Speed']<=12),: ]
idealvacation_df= idealvacation_df.dropna(how='any')
len(idealvacation_df)

17

### Hotel Map



In [71]:
# Store into variable named `hotel_df`.
hotel_df=idealvacation_df.loc[:,["City", "Country","Lat","Lng"]]
hotel_df

,City,Country,Lat,Lng
146,Kisi,NG,9.0833,3.8500
148,Tanout,NE,14.9709,8.8879
156,Ormara,PK,25.2088,64.6357
180,Kaduna,NG,10.5222,7.4383
186,Misratah,LY,32.3754,15.0925
216,Jiwani,PK,25.0500,61.7417
276,Zemio,CF,5.0314,25.1361
341,Whitianga,NZ,-36.8333,175.7000
384,Springbok,ZA,-29.6643,17.8865
390,Zliten,LY,32.4674,14.5687


In [72]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
146,Kisi,NG,9.0833,3.8500,
148,Tanout,NE,14.9709,8.8879,
156,Ormara,PK,25.2088,64.6357,
180,Kaduna,NG,10.5222,7.4383,
186,Misratah,LY,32.3754,15.0925,
216,Jiwani,PK,25.0500,61.7417,
276,Zemio,CF,5.0314,25.1361,
341,Whitianga,NZ,-36.8333,175.7000,
384,Springbok,ZA,-29.6643,17.8865,
390,Zliten,LY,32.4674,14.5687,


In [73]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": gkey
}
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    hotel_data=requests.get(base_url, params=params).json()
    #print(json.dumps(hotel_data, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [74]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
146,Kisi,NG,9.0833,3.8500,BASUAK HOTELS
148,Tanout,NE,14.9709,8.8879,
156,Ormara,PK,25.2088,64.6357,Alqadri hotel zero point ormara
180,Kaduna,NG,10.5222,7.4383,castle de white house hotel kaduna
186,Misratah,LY,32.3754,15.0925,نزل طريق المطار - مصراتة
216,Jiwani,PK,25.0500,61.7417,Victoria Hut
276,Zemio,CF,5.0314,25.1361,
341,Whitianga,NZ,-36.8333,175.7000,Whitianga Hotel
384,Springbok,ZA,-29.6643,17.8865,Springbok Inn by Country Hotels
390,Zliten,LY,32.4674,14.5687,Zliten Hotel


In [75]:
hotel_df=hotel_df.loc[hotel_df['Hotel Name']!='']
hotel_df


,City,Country,Lat,Lng,Hotel Name
146,Kisi,NG,9.0833,3.8500,BASUAK HOTELS
156,Ormara,PK,25.2088,64.6357,Alqadri hotel zero point ormara
180,Kaduna,NG,10.5222,7.4383,castle de white house hotel kaduna
186,Misratah,LY,32.3754,15.0925,نزل طريق المطار - مصراتة
216,Jiwani,PK,25.0500,61.7417,Victoria Hut
341,Whitianga,NZ,-36.8333,175.7000,Whitianga Hotel
384,Springbok,ZA,-29.6643,17.8865,Springbok Inn by Country Hotels
390,Zliten,LY,32.4674,14.5687,Zliten Hotel
444,Guerrero Negro,MX,27.9769,-114.0611,Hotel TerraSal
465,NeuquÃ©n,AR,-38.9516,-68.0591,Casino Magic Hotel & Casino


In [76]:
#Plot markers on top of the heatmap.

In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [79]:
fig = gmaps.figure()

fig.add_layer(humidity_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))